In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns 
import matplotlib.pyplot as plt 


In [ ]:
url='../input/company-bankruptcy-prediction/'

df=pd.read_csv(url+'data.csv')
df.head()
print(df.columns)

In [ ]:
df.shape

# unbalanced data
on constate qu'on a une base de données désiquilibres classe faillite 220 non faillite 6599 ==> on va appliquer svm mono classe

In [ ]:
label=df['Bankrupt?']
print(label)
label.value_counts()


In [ ]:
data=df.drop(['Bankrupt?'],axis=1)
print(data)

# reduction des demension 
on ne peut pas utiliser lda dans ce probléme car ona une classification binaire donc 2 classe (impossible d'appliquer lda puisque nbre de features in lda reduction égale a nbre de classe -1 et donc 2-1=1 )
dans ce cas on va utiliser pca et on va initialiser nbre de components avec un valeur comprise entre 0 et min(6819, 96)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
Transf_pca = PCA(n_components= 10)
datanew = Transf_pca.fit_transform(data)
print(datanew)
datanew.shape
#datanew shape(6819,10)

# Classification SVM monoclass

In [ ]:
label=pd.DataFrame(label)
label.shape
newbase=np.concatenate((datanew,label),axis=1)
print(newbase)
newbase.shape
newbase=pd.DataFrame(newbase)
print(newbase.columns)


diviser la base en 2 partie : partie qui inclus les data qui on target 1 et partie inclus target 0 

In [ ]:
dataset_selected1=newbase.loc[newbase[10].isin([1])]
dataset_selected0=newbase.loc[newbase[10].isin([0])]


#on va diviser dataset_selected1 en 2 partie data1 et label1
label1=dataset_selected1[10]
data1=dataset_selected1.drop([10],axis=1)

#on va diviser dataset_selected0 en 2 partie data0 et label0
label0=dataset_selected0[10]
#remplacer 0 par -1
label0=label0-1
data0=dataset_selected0.drop([10],axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
x_train1,x_test1,y_train1,y_test1=train_test_split(data1,label1,test_size=0.33,random_state=0)
from sklearn import svm
model=svm.OneClassSVM(kernel='rbf',nu=1,gamma=0.00001)
import time
debut=time.time()
model.fit(x_train1)
fin=time.time()-debut

In [ ]:
import numpy as np
data_tesst=np.concatenate((x_test1,data0),axis=0)
label_tesst=np.concatenate((y_test1,label0),axis=0)
pred=model.predict(data_tesst)


# calculer accurency

In [ ]:
from sklearn.metrics import accuracy_score
ACC=accuracy_score(label_tesst,pred)*100
print('the accurency score is :')
print(ACC)

# report de prediction

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(label_tesst,pred))


# Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
CM=confusion_matrix(label_tesst,pred)
print(CM)
#heatmap de confusion matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
class_names=[0,1] # name of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
# create heatmap
# CM1=pd.DataFrame(CM)
# print(CM1)
sns.heatmap(pd.DataFrame(CM), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
#plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

# OverSampling

on peut transformer unbalanced data to balanced data 

In [ ]:
print(newbase)
label=newbase[10]
data=newbase.drop([10],axis=1)
label.value_counts()

In [ ]:
from imblearn.over_sampling import RandomOverSampler
import imblearn
print(imblearn.__version__)

# define oversampling strategy
oversample = RandomOverSampler(sampling_strategy='minority')

# define oversampling strategy
oversample = RandomOverSampler(sampling_strategy=1.0)

# fit and apply the transform
X_over, y_over = oversample.fit_resample(data, label)
#compter combien de 1 et de 0 dans dataset
label=y_over
data=X_over
label =pd.DataFrame(label)
label.value_counts()

ona équilibre dataset

# Decision tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
dtree = DecisionTreeClassifier()
x_train,x_test,y_train,y_test = train_test_split(data,label,test_size = 0.3,random_state = 0)
import time
debut=time.time()
dtree.fit(x_train,y_train)
fin=time.time()-debut
prediction = dtree.predict(x_test)

# Accurency

In [ ]:
from sklearn.metrics import accuracy_score
ACC=accuracy_score(y_test,prediction)*100
print('With decision tree accuracy is: ',ACC) # accuracy

# Report

In [ ]:
from sklearn.metrics import classification_report
Class=classification_report(y_test,prediction)
print(Class)

# Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
CM=confusion_matrix(y_test,prediction)
print(CM)

#heatmap de confusion matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
class_names=[0,1] # name of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
# create heatmap
# CM1=pd.DataFrame(CM)
# print(CM1)
sns.heatmap(pd.DataFrame(CM), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
#plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')


# Roc curve

In [ ]:
from sklearn.metrics import roc_curve,auc
fp, tp, thresholds=roc_curve(y_test,prediction,pos_label=1)
print(fp, tp)
AUC=auc(fp, tp)*100
print(AUC)

#tracer tp en fonction de fp
import matplotlib.pyplot as plt
plt.plot(fp, tp, color='blue',label = 'AUC = %0.2f' % AUC)
plt.title('Receiver Operating Characteristic')
#plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % AUC)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()



===> on constate que auc=98% donc l'algorithme est trés performant

In [ ]:
from sklearn.tree import plot_tree
#affichage plus grand pour une meilleure lisibilité
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
plot_tree(dtree)
plt.show()

# F1_score

In [ ]:
from sklearn.metrics import f1_score
f1score=f1_score(y_test,prediction)*100
print('f1_score is: ',f1score) 

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
rf = RandomForestClassifier(criterion = 'gini')
x_train,x_test,y_train,y_test = train_test_split(data,label,test_size = 0.3,random_state =1)
import time
debut=time.time()
rf.fit(x_train,y_train)
fin=time.time()-debut

prediction = rf.predict(x_test)

# accurency

In [ ]:
from sklearn.metrics import accuracy_score
ACC=accuracy_score(y_test,prediction)*100
print('With random forest accuracy is: ',ACC) # accuracy

# report

In [ ]:
from sklearn.metrics import classification_report
Class=classification_report(y_test,prediction)
print(Class)

# confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
CM=confusion_matrix(y_test,prediction)
print(CM)

#heatmap de confusion matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
class_names=[0,1] # name of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
# create heatmap
# CM1=pd.DataFrame(CM)
# print(CM1)
sns.heatmap(pd.DataFrame(CM), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
#plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')


# Roc curve

In [ ]:
from sklearn.metrics import roc_curve,auc
fp, tp, thresholds=roc_curve(y_test,prediction,pos_label=1)
print(fp, tp)
AUC=auc(fp, tp)*100
print(AUC)

#tracer tp en fonction de fp
import matplotlib.pyplot as plt
plt.plot(fp, tp, color='blue',label = 'AUC = %0.2f' % AUC)
plt.title('Receiver Operating Characteristic')
#plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % AUC)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()



In [ ]:
from sklearn.metrics import f1_score
f1score=f1_score(y_test,prediction)*100
print('f1_score is: ',f1score) 